In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import NuLattice.smearedInteraction as smearI
import NuLattice.CCM.coupled_cluster as ccm
import NuLattice.references as ref
import NuLattice.FCI.few_body_diagonalization as fbd
import NuLattice.lattice as lat
from scipy.sparse.linalg import eigsh as arpack_eigsh

In [6]:
myL= 4
bpi = 0.7
sL = 0.08
sNL = 0.08
a = 1.0 / 100.0
at = 1.0 / 1e8
c0 = -1.85e-05/ (a ** 2)
tKin, inter = smearI.get_full_int(myL, bpi, c0, sL, sNL, a, at, 2)
holes = ref.ref_2H_gs
no2b_stuff = ccm.get_norm_ordered_ham(myL, holes, tKin, inter)

In [7]:
# Compute the deuteron
# additive quantum numbers
numpart=2 # number of nucleons
tz = 0    # twice the value of isospin projection
sz = 2    # twice the value of spin projection
my_basis = lat.get_sp_basis(myL)
nstat =  len(my_basis)

# get two-body basis as a dictionary for lookup
H2_lookup = fbd.get_many_body_states(my_basis, numpart, total_tz=tz, total_sz=sz)
print("matrix dimension:", len(H2_lookup))

# make scipy.sparse.csr_matrix for kinetic energy 
T2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, tKin, nstat)
print("kinetic energy done")

# make scipy.sparse.csr_matrix for 2-body interactions 
V2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, inter, nstat)
print("2-body interaction done")

# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + V2_csr_mat

# compute lowest eigenvalue(s)
k_eig=10  # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energies (MeV):", vals / a)

matrix dimension: 4096
kinetic energy done
2-body interaction done
Energies (MeV): [-3.60895936 -3.60895936 -0.39343633 -0.36696966 -0.36696966 -0.36696966
 -0.36696966 -0.36696966 -0.36696966 -0.36696966]


In [62]:
corrEn, t1, t2 = ccm.ccsd_solver(no2b_stuff[1], no2b_stuff[2], 
                            eps = 1e-8, maxSteps = 500, max_diis = 10, mixing = 0.5,
                            sparse=True, verbose=True, delta = -10)
ccsdEn = corrEn + no2b_stuff[0]
print(f'Energy: {ccsdEn / at}')

Step 0: -1.075258189937382e-06
Step 1: 0.0012953670614850098 difference = 1.0008300799224468
Step 2: 0.0005971898369319832 difference = 1.169104330609272
Step 3: 0.0021600952435877336 difference = 0.7235354141421553
Step 4: 0.0020574187334617745 difference = 0.049905499768244835
Step 5: 0.0021941882242117908 difference = 0.062332615425072485
Step 6: 0.0025740476136065825 difference = 0.14757279056798733
Step 7: 0.003203687301119219 difference = 0.19653593760310809
Step 8: -0.0076503207251407135 difference = 1.4187650970751027
Step 9: 3.879776255021323 difference = 1.0019718458545745
Step 10: -3.2435807490637423 difference = 2.196139869846378
Step 11: 0.002138672497837957 difference = 1517.6327487461347
Step 12: 0.0012817872861015619 difference = 0.6685081222349557
Step 13: 0.0036604295261382083 difference = 0.6498259898329852
Step 14: 0.006926171690146123 difference = 0.4715075383785377
Step 15: -0.0006765588321140242 difference = 11.237353148585926
Step 16: -0.016943748901589853 diffe

KeyboardInterrupt: 